In [3]:
import monai
import os
import torch
import numpy as np
import yaml
import datetime
import wandb
from Pytorch_monai.secrets import wandbkey
import Pytorch_monai.Model_and_transforms as MF
import Pytorch_monai.Utils as Utils
import json

### intialize from config file
with open('./config.yaml', 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

# train_label_file = cfg['training']['train_label_file']
train_label_file = '/home/soenke/Internship/data/DATA/labels.txt'
train_labelmap_file = cfg['training']['label_map_file']
x_image_size = cfg['data_preparation']['image_size_x']
y_image_size = cfg['data_preparation']['image_size_y']
output_folder = cfg['training']['output_folder']
batch_size = cfg['network']['batch_size']
nb_epoch = cfg['network']['nb_epoch']

## setup gpu and model name
gpu = Utils.chooseDevice()
now = str(datetime.datetime.now()).replace(' ', '_')
model_name = 'DDS_model_epochs' + str(nb_epoch) + '_time_' + now

##label map
with open(train_labelmap_file) as labelmap:
    label_map = json.load(labelmap)

# load imagefilenames and onehot encoded labels
train_image_IDs, train_image_labels, N_train_classes, extra_inputs = Utils.load_labels(train_label_file)
print("Detected %d classes in training data" % N_train_classes)

#initialize monai transforms
trainTransforms = monai.transforms.Compose(
    [
        monai.transforms.LoadImaged(keys=['image'],image_only=False,reader='NibabelReader',ensure_channel_first=False),
        monai.transforms.EnsureTyped(keys=['image']),
        monai.transforms.EnsureChannelFirstd(keys=['image'])
    ]
)
valTransforms = monai.transforms.Compose(
    [
        monai.transforms.LoadImaged(keys=['image'],image_only=False,reader='NibabelReader',ensure_channel_first=False),
        monai.transforms.EnsureTyped(keys=['image']),
        monai.transforms.EnsureChannelFirstd(keys=['image'])
    ]
)

#create data dicitionaries and data loader to which monai transforms can be applied (transforms stored in Model_and_transforms.py)
train_data_dict = [{"image":image_name,"label":label} for image_name, label in zip(train_image_IDs,train_image_labels)]
val_data_dict = train_data_dict[-10:]
train_data_dict = train_data_dict[:-10] #this can be optimized to shuffle beforehand for example
train_ds = monai.data.CacheDataset(data=train_data_dict,transform=trainTransforms,cache_rate=1,num_workers=4)
val_ds = monai.data.CacheDataset(data=val_data_dict,transform=valTransforms,cache_rate=1,num_workers=4)
train_loader = monai.data.DataLoader(train_ds,batch_size=1,shuffle=True,num_workers=0)
val_loader = monai.data.DataLoader(val_ds,batch_size=1,shuffle=True,num_workers=0)

Detected 1 classes in training data


Loading dataset: 100%|██████████| 10/10 [00:00<00:00, 463.05it/s]


In [4]:
testImg = monai.utils.first(train_loader)
testImg['image'].shape
#|testImg['image_meta_dict']

torch.Size([1, 1, 256, 256])

In [5]:
testImg['image_meta_dict']

{'sizeof_hdr': tensor([348], dtype=torch.int32),
 'extents': tensor([0], dtype=torch.int32),
 'session_error': tensor([0], dtype=torch.int16),
 'dim_info': tensor([0], dtype=torch.uint8),
 'dim': tensor([[  2, 256, 256,   1,   1,   1,   1,   1]], dtype=torch.int16),
 'intent_p1': tensor([0.]),
 'intent_p2': tensor([0.]),
 'intent_p3': tensor([0.]),
 'intent_code': tensor([0], dtype=torch.int16),
 'datatype': tensor([16], dtype=torch.int16),
 'bitpix': tensor([32], dtype=torch.int16),
 'slice_start': tensor([0], dtype=torch.int16),
 'pixdim': tensor([[1., 1., 1., 1., 1., 1., 1., 1.]]),
 'vox_offset': tensor([0.]),
 'scl_slope': tensor([nan]),
 'scl_inter': tensor([nan]),
 'slice_end': tensor([0], dtype=torch.int16),
 'slice_code': tensor([0], dtype=torch.uint8),
 'xyzt_units': tensor([0], dtype=torch.uint8),
 'cal_max': tensor([0.]),
 'cal_min': tensor([0.]),
 'slice_duration': tensor([0.]),
 'toffset': tensor([0.]),
 'glmax': tensor([0], dtype=torch.int32),
 'glmin': tensor([0], dtype=